SIDE NOTE: would love to get any critiques, hints, tips, oberservations on any level not just on ML (but obviously ML as the priority).  Beginner Python learner (though I took the Full Stack Course) so any suggestions there are welcome.  Thanks.

Will need to import these libs plus setting some global variables

In [1]:
import sys
import os
import numpy as np
import renders as rs
import pandas as pd 
import matplotlib.pyplot as plt
from IPython.display import display # Allows the use of display() for DataFrames
from sklearn.externals import joblib
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
import matplotlib.cm as cm
# Show matplotlib plots inline (nicely formatted in the notebook)
%matplotlib inline

# Might need to change the path of the included libraries.
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/match_stats.py')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/form_model.py')
sys.path.append('/anaconda/envs/stats/lib/python3.5/site-packages')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/model_libs.py')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/form_data.py')
sys.path.append('/Users/senzari/Machine_Learning/stats')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats')
# print(sys.path)
os.chdir('/Users/senzari/Machine_Learning/stats/src')
#print(os.getcwd())

from stats import form_data, match_stats, model_libs, form_model, predict_matches

# Variables
round_number = 27 # for MLS only
target_col = 'points'
ignore_cols = ['match_id', 'team_id', 'team_name', 'opp_id', 'opp_name', 'scheduled', 'games_played', 'round']
sub_cols = ['current_formation', 'avg_goals_against', 'goal_diff', 'win_percentage', 'sos',
           'opp_win_percentage', 'opp_sos', 'current_team_yellow_cards', 'current_team_corner_kicks', 'current_team_first_half_goals', 'current_team_sec_half_goals', 
           'opp_team_yellow_cards', 'opp_team_corner_kicks', 'opp_team_first_half_goals', 'opp_team_sec_half_goals']

all_models = ['log', 'svc', 'gmm', 'knn', 'gnb', 'randomForest']

""" this variable 'testing' should be false if using CSV's and not pulling from the database. """
testing = False

INITIALIZED...


In [2]:
data_csv = 'raw_data_squared.csv'

if testing:
    raw_data = form_data.run_data()
    raw_data.to_csv(data_csv)
    print("Raw Data Saved to CSV")
else:
    #Reading in a CSV adds the first index column
    raw_data = pd.read_csv(data_csv)
    raw_data = raw_data.drop(raw_data.columns[[0]], axis=1)

pd.set_option("display.max_columns", 85)
print('Data Loaded...')
print("Dataset size :: {}".format(raw_data.shape))
display(raw_data.head())

Data Loaded...
Dataset size :: (634, 21)


,match_id,team_id,team_name,opp_id,opp_name,scheduled,round,games_played,is_home,current_formation,goals_for,goals_allowed,opp_goals_allowed,goal_efficiency,opp_defensive_goal_efficiency,ratio_of_attacks,opp_ratio_of_attacks,ratio_ball_safe_to_dangerous_attacks,opp_ratio_ball_safe_to_dangerous_attacks,goals,points
0,249,21,FC Dallas,33,DC United,2016-03-26 21:30:00,4,3,0,4-2-3-1,4,5,5,0.114286,0.750000,0.587097,0.524917,1.210938,1.037931,3,3
1,255,21,FC Dallas,39,Columbus Crew,2016-04-03 01:00:00,5,3,1,4-2-3-1,5,5,4,0.156250,0.818182,0.502976,0.538136,1.183099,0.874074,1,1
2,265,21,FC Dallas,32,San Jose Earthquakes,2016-04-10 01:00:00,6,3,1,4-4-2,6,1,5,0.133333,0.821429,0.495575,0.504323,1.185315,1.074303,2,1
3,272,21,FC Dallas,30,Portland Timbers,2016-04-14 02:30:00,7,3,0,4-4-2,6,3,7,0.146341,0.695652,0.467742,0.609907,1.148148,1.247104,3,3
4,282,21,FC Dallas,27,Sporting Kansas City,2016-04-17 23:00:00,7,3,1,4-4-2,6,3,2,0.146341,0.942857,0.467742,0.534946,1.148148,1.053824,2,3


## FORMATTING

Converting Goals to our binary classification (0-1) and (2+) and also converting the sos and rpi and implementing the team to a particular rpi quartile

In [7]:
# Helper Function - Removes Columns to Ignore and Splits the Target Column
def split_target(data):
    td = model_libs._clone_and_drop(data, ignore_cols)
    (y, X) = model_libs._extract_target(td, target_col)
    return X, y

""" Need to do some formatting of the Data before we run the models"""
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1000)

rankings_data = raw_data.copy()
rankings_data["offensive_ranking"] = pd.Series(None, index=rankings_data.index)
rankings_data["opp_defensive_ranking"] = pd.Series(None, index=rankings_data.index)

leagues = model_libs.get_leagues_country_codes()
#leagues = { "epl": 'ENG' }
teams = form_data.get_teams()
league_rounds = model_libs.get_leagues_rounds()
test = False
if test:
    """ Going through each League"""
    for key, value in leagues.iteritems():
        print(key)
        country_code = leagues[key]
        round_num = league_rounds[key]
        #round_num = 6
        teams_in_league = teams[teams["country_code"] == country_code]
        """ Looping through the Rounds """
        for i in range(4, round_num):
            print("ROUND :: {} ".format(i))
            offensive_rankings = form_data.get_rankings(teams_in_league, i, "offensive")
            rankings = model_libs.quartile_list(offensive_rankings, True)
            offensive_rankings["offensive_rankings_quartiled"] = rankings
            print("Finished with Offensive Rankings")
            #print(offensive_rankings)

            defensive_rankings = form_data.get_rankings(teams_in_league, i, "defensive")
            rankings = model_libs.quartile_list(defensive_rankings, False)
            defensive_rankings["defensive_rankings_quartiled"] = rankings
            print("Finished with Defensive Rankings")
            #print(defensive_rankings)

            """ Loop through each Team in the League for that round and assign an Offensive Rank """
            for key, team in teams_in_league.iterrows():
                
                ''' If the team is the team_id then put in their offensive ranking for that game '''
                offensive_rank = offensive_rankings.loc[offensive_rankings[0] == team['id'], "offensive_rankings_quartiled"]
                idx = rankings_data.loc[(rankings_data["team_id"] == team["id"]) 
                        & (rankings_data["round"] == (i)), "offensive_ranking"].index

                rankings_data.set_value(idx, "offensive_ranking", offensive_rank.values[0])
                ''' If the team is the opp then put in their defensive ranking for that game '''
                defensive_rank = defensive_rankings.loc[defensive_rankings[0] == team['id'], "defensive_rankings_quartiled"]
                opp_idx = rankings_data.loc[(rankings_data["opp_id"] == team["id"]) 
                        & (rankings_data["round"] == (i))].index

                rankings_data.set_value(opp_idx, "opp_defensive_ranking", defensive_rank.values[0])
                
                rankings_data.to_csv('rankings_data.csv')
                
else:
    
    rankings_data = pd.read_csv('rankings_data.csv')
    rankings_data = rankings_data.drop(rankings_data.columns[[0]], axis=1)
                          
""" Formatting data to convert goals scored to the correct category"""
# Not using points as a target for this version, using goals
rankings_data = rankings_data.drop('goals', 1)
#rankings_data['converted_points'] = rankings_data.apply(lambda row: model_libs.set_points(row['points']), axis=1)
rankings_data = rankings_data.drop(ignore_cols + ['current_formation'], 1)

print('Data Loaded...')

Data Loaded...


### RUNNING CLASSIFICATION MODEL ON IF TEAMS SCORE 0-1 OR 1-2 ON GAMES

In [20]:
def run_features(data, drop_data, target, models):
    
    new_data = data.drop(drop_data, axis=1)
    
    #display(new_data.head())
    
    (y, X) = model_libs._extract_target(new_data, target)
    
    models = form_model.train_models(round_number, X, y, models)
    
    return models

#rankings_data = rankings_data.drop(['goals_for', 'goals_allowed', 'opp_goals_allowed'], 1)

#models_test_1 = run_features(rankings_data, [], 'converted_points', ["knn"])

(rankings_y, rankings_X) = model_libs._extract_target(rankings_data, 'points')

classif = OneVsRestClassifier(SVC(kernel='rbf', probability=True, class_weight='auto'))
classif.fit(rankings_X, rankings_y)

def check_accuracy(model, data_X):
    actual_y = pd.DataFrame(rankings_y.values, columns=['actual'])
    predictions = pd.DataFrame(model.predict(data_X), columns=['predictions'])
    preds = pd.concat([predictions, actual_y], axis=1)
    preds['diff'] = preds.apply(lambda r: model_libs.predictions_diff(r['predictions'], r['actual']), axis=1)
    display(preds)
    accuracy = np.divide(preds['diff'].sum(), float(len(preds['diff'])))
    print(accuracy)

#for m in models_test_1:
check_accuracy(classif, rankings_X)

//anaconda/lib/python2.7/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)
//anaconda/lib/python2.7/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)
//anaconda/lib/python2.7/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)


,predictions,actual,diff
0,0,3,0
1,3,1,0
2,3,1,0
3,1,3,0
4,3,3,1
5,1,0,0
6,1,0,0
7,0,0,1
8,3,3,1
9,3,3,1


0.545741324921


In [ ]:
def train_tuned_models(round_num, X, y):
    for i in all_models:
        models = form_model.build_tuned_model(X, y, i)
        
    return models
        
tuned_models = train_tuned_models(round_number, classifier_X, classifier_y)

In [ ]:
log_model = form_model.build_tuned_model(rankings_X, rankings_y, 'log')

for m in log_model:
    print(m)
    check_accuracy(m, rankings_X)

In [ ]:
random_forest_model = form_model.build_tuned_model(rankings_X, rankings_y, 'randomForest')

for m in random_forest_model:
    print(m)
    check_accuracy(m, rankings_X)

In [ ]:
knn_model = form_model.build_tuned_model(rankings_X, rankings_y, 'knn')

for m in knn_model:
    check_accuracy(m, rankings_X)

In [ ]:
gnb_model = form_model.build_tuned_model(rankings_X, rankings_y, 'gnb')

for m in gnb_model:
    print(m)
    check_accuracy(m, rankings_X)

In [24]:
svc_model = form_model.build_tuned_model(rankings_X, rankings_y, 'svc')

for m in svc_model:
    print(m)
    check_accuracy(m, rankings_X)

Training and Tuning SVC Model
[ 0.37795276  0.47826087]
Accuracy: 0.43 (+/- 0.10)
Finished SVC Modeling
GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'kernel': ['rbf'], 'C': [1, 10, 100]}],
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)


,predictions,actual,diff
0,0,3,0
1,3,1,0
2,3,1,0
3,0,3,0
4,3,3,1
5,0,0,1
6,0,0,1
7,0,0,1
8,3,3,1
9,3,3,1


0.531545741325


In [ ]:
gmm_model = form_model.build_tuned_model(rankings_X, rankings_y, 'gmm')

for m in gmm_model:
    print(m)
    check_accuracy(m, rankings_X)

Cross Validating the SVC model with the PCA data to help prevent overfitting

In [ ]:
prediction_models = form_model.load_models(['knn', 'svc', 'randomForest', 'gnb', 'gmm', 'log'])

In [9]:
print('Upcoming matches')
#upcoming_matches, match_details = predict_matches.get_upcoming_matches()
#upcoming_matches.to_csv('upcoming_matches.csv')
upcoming_matches = pd.read_csv('upcoming_matches.csv')
upcoming_matches = upcoming_matches.drop(upcoming_matches.columns[[0]], axis=1)
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)
upcoming_data = predict_matches.predictions(upcoming_matches)


print('Loaded...')
upcoming_data["offensive_ranking"] = pd.Series(None, index=rankings_data.index)
upcoming_data["opp_defensive_ranking"] = pd.Series(None, index=rankings_data.index)

leagues = model_libs.get_leagues_country_codes()
teams = form_data.get_teams()
league_rounds = model_libs.get_leagues_rounds()
test = False
if test:
    """ Going through each League"""
    for key, value in leagues.iteritems():
        print(key)
        country_code = leagues[key]
        round_num = league_rounds[key]
        teams_in_league = teams[teams["country_code"] == country_code]
        
        print("ROUND :: {} ".format(round_num))
        offensive_rankings = form_data.get_rankings(teams_in_league, round_num, "offensive", True)
        print(offensive_rankings)
        rankings = model_libs.quartile_list(offensive_rankings, True)
        offensive_rankings["offensive_rankings_quartiled"] = rankings
        #print(offensive_rankings)
        print("Finished with Offensive Rankings")

        defensive_rankings = form_data.get_rankings(teams_in_league, round_num, "defensive", True)
        rankings = model_libs.quartile_list(defensive_rankings, False)
        defensive_rankings["defensive_rankings_quartiled"] = rankings
        #print(defensive_rankings)
        print("Finished with Defensive Rankings")

        """ Loop through each Team in the League for that round and assign an Offensive Rank """
        for key, team in teams_in_league.iterrows():
                
            ''' If the team is the team_id then put in their offensive ranking for that game '''
            offensive_rank = offensive_rankings.loc[offensive_rankings[0] == team['id'], "offensive_rankings_quartiled"]
            idx = upcoming_data.loc[(upcoming_data["team_id"] == team["id"]) 
                    & (upcoming_data["round"] == (round_num)), "offensive_ranking"].index
            upcoming_data.set_value(idx, "offensive_ranking", offensive_rank.values[0])
            
            ''' If the team is the opp then put in their defensive ranking for that game '''
            defensive_rank = defensive_rankings.loc[defensive_rankings[0] == team['id'], "defensive_rankings_quartiled"]
            opp_idx = upcoming_data.loc[(upcoming_data["opp_id"] == team["id"]) 
                    & (upcoming_data["round"] == (round_num))].index

            upcoming_data.set_value(opp_idx, "opp_defensive_ranking", defensive_rank.values[0])
                
            # upcoming_data.to_csv('upcoming_ranking_data.csv')
                
else:
    
    upcoming_data = pd.read_csv('upcoming_ranking_data.csv')
    upcoming_data = upcoming_data.drop(upcoming_data.columns[[0]], axis=1)

display(upcoming_data.head())

Upcoming matches
Loaded...


,match_id,team_id,team_name,opp_id,opp_name,scheduled,round,games_played,is_home,current_formation,goals_for,goals_allowed,opp_goals_allowed,goal_efficiency,opp_defensive_goal_efficiency,ratio_of_attacks,opp_ratio_of_attacks,ratio_ball_safe_to_dangerous_attacks,opp_ratio_ball_safe_to_dangerous_attacks,goals,points,offensive_ranking,opp_defensive_ranking
0,900,21,FC Dallas,23,Real Salt Lake,2016-09-25 01:30:00,28,3,0,4-2-3-1,5,4,5,0.128205,0.782609,0.541528,0.396947,1.264706,0.970370,0,0,1.00000,1.00000
1,896,22,Colorado Rapids,28,Vancouver Whitecaps FC,2016-09-24 23:00:00,28,3,0,4-2-3-1,1,2,3,0.041667,0.896552,0.482993,0.591973,1.050000,1.150000,0,0,0.00000,0.66666
2,900,23,Real Salt Lake,21,FC Dallas,2016-09-25 01:30:00,28,3,1,4-3-3,3,5,4,0.088235,0.800000,0.561934,0.480263,1.153310,0.924012,0,0,0.33333,1.00000
3,894,24,New York City FC,40,Chicago Fire,2016-09-24 00:00:00,28,3,1,4-3-3,6,7,4,0.214286,0.888889,0.480263,0.490506,0.924012,1.029316,0,0,0.33333,0.33333
4,895,25,Philadelphia Union,34,Toronto FC,2016-09-24 21:00:00,28,3,0,4-2-3-1,2,6,5,0.064516,0.791667,0.422713,0.505119,1.096886,1.042705,0,0,0.33333,1.00000


In [13]:
formatted_upcoming_data = upcoming_data.drop(ignore_cols + ['current_formation', 'goals', 'points'], axis=1)
display(formatted_upcoming_data.head())

,is_home,goals_for,goals_allowed,opp_goals_allowed,goal_efficiency,opp_defensive_goal_efficiency,ratio_of_attacks,opp_ratio_of_attacks,ratio_ball_safe_to_dangerous_attacks,opp_ratio_ball_safe_to_dangerous_attacks,offensive_ranking,opp_defensive_ranking
0,0,5,4,5,0.128205,0.782609,0.541528,0.396947,1.264706,0.970370,1.00000,1.00000
1,0,1,2,3,0.041667,0.896552,0.482993,0.591973,1.050000,1.150000,0.00000,0.66666
2,1,3,5,4,0.088235,0.800000,0.561934,0.480263,1.153310,0.924012,0.33333,1.00000
3,1,6,7,4,0.214286,0.888889,0.480263,0.490506,0.924012,1.029316,0.33333,0.33333
4,0,2,6,5,0.064516,0.791667,0.422713,0.505119,1.096886,1.042705,0.33333,1.00000


In [27]:
""" Models we'll use to predict on upcoming matches """
# pca_svc_model, knn_model, random_forest_model

# This is all the X values
formatted_upcoming_data

classif_preds = classif.predict(formatted_upcoming_data)
print(classif_preds)

svc_preds = svc_model[0].predict(formatted_upcoming_data)
print(svc_preds)



[ 0.  0.  3.  3.  0.  1.  0.  3.  0.  0.  3.  3.  1.  1.  1.  1.  0.  3.
  3.  1.  3.  1.  3.  1.  3.  0.  0.  0.  0.  0.  3.  1.  1.  0.  3.  3.
  0.  1.  1.  3.  1.  0.  0.  0.  0.  1.  3.  3.  3.  1.  0.  0.  1.  0.
  1.  3.  0.  3.  0.  0.  0.  3.  3.  3.  3.  1.  3.  3.  0.  3.  0.  1.
  0.  1.  0.  1.  0.  1.  3.  0.  0.  3.  0.  0.  3.  3.  3.  0.  3.  0.
  3.  3.  3.  0.  3.  1.  1.  1.]
[ 0.  0.  3.  3.  0.  0.  0.  3.  0.  0.  3.  3.  1.  3.  1.  0.  0.  3.
  3.  0.  3.  0.  3.  1.  3.  3.  0.  0.  0.  0.  3.  0.  3.  0.  3.  3.
  0.  3.  0.  3.  3.  0.  0.  0.  0.  1.  3.  3.  3.  3.  0.  0.  1.  0.
  3.  3.  0.  3.  0.  0.  0.  3.  3.  3.  3.  3.  3.  3.  0.  3.  0.  3.
  0.  1.  0.  1.  0.  3.  3.  0.  0.  3.  0.  0.  0.  3.  3.  0.  3.  0.
  3.  3.  3.  0.  3.  1.  0.  1.]


In [29]:
columns = ['team_name', 'opp_name', 'scheduled']
# Remove all columns except the ones above
upcoming_matches = upcoming_data[columns]

#random_preds = pd.Series(np.random.randint(2, size=len(upcoming_matches.index)), upcoming_matches.index)

# Add predictions to the end of that DF
results = pd.DataFrame({'Classif': classif_preds, 'svc': svc_preds})

upcoming_matches = pd.concat([upcoming_matches, results], axis = 1)
display(upcoming_matches.head())
reordered_matches = pd.DataFrame([])

for rows in upcoming_matches.iterrows():
    for i in upcoming_matches['team_name']:
        if rows[1]['opp_name'] == i:
            reordered_matches = reordered_matches.append(rows[1])
            reordered_matches = reordered_matches.append(upcoming_matches[upcoming_matches['team_name'].isin([i])])

reordered_matches = reordered_matches.drop_duplicates() 
columns = ['scheduled', 'team_name', 'opp_name', 'Classif', 'svc']
reordered_matches = reordered_matches[columns]
reordered_matches.to_csv('predictions_on_ranked_data_points_onevsall.csv')
print('Prediction CSV saved')

,team_name,opp_name,scheduled,Classif,svc
0,FC Dallas,Real Salt Lake,2016-09-25 01:30:00,0,0
1,Colorado Rapids,Vancouver Whitecaps FC,2016-09-24 23:00:00,0,0
2,Real Salt Lake,FC Dallas,2016-09-25 01:30:00,3,3
3,New York City FC,Chicago Fire,2016-09-24 00:00:00,3,3
4,Philadelphia Union,Toronto FC,2016-09-24 21:00:00,0,0


Prediction CSV saved


In [31]:
actual_data = pd.read_csv('predictions_on_ranked_data_points_onevsall.csv')
actual_data = actual_data.drop(actual_data.columns[[0]], axis=1)
actual_data = actual_data.drop(['Classif'], axis=1)
display(actual_data)
#actual_data.to_csv('actual_data_analysis.csv')
actual_data['diff1'] = actual_data.apply(lambda r: model_libs.predictions_diff(r['actual'], r['svc']), axis=1)
accuracy = np.divide(actual_data['diff1'].sum(), float(len(actual_data['diff1'])))
print(accuracy)

,scheduled,team_name,opp_name,svc,actual
0,9/25/16 1:30,FC Dallas,Real Salt Lake,0,1
1,9/25/16 1:30,Real Salt Lake,FC Dallas,3,1
2,9/24/16 23:00,Colorado Rapids,Vancouver Whitecaps FC,0,1
3,9/24/16 23:00,Vancouver Whitecaps FC,Colorado Rapids,3,1
4,9/24/16 0:00,New York City FC,Chicago Fire,3,3
5,9/24/16 0:00,Chicago Fire,New York City FC,0,0
6,9/24/16 21:00,Philadelphia Union,Toronto FC,0,1
7,9/24/16 21:00,Toronto FC,Philadelphia Union,3,1
8,9/25/16 20:00,LA Galaxy,Seattle Sounders,0,0
9,9/25/16 20:00,Seattle Sounders,LA Galaxy,0,3


0.540816326531


In [ ]:
abbreviated_data = actual_data.drop(["KNN", "RandomForest", "GNB", "GMM", "random"], axis=1)
print(abbreviated_data.shape)
display(abbreviated_data)